# KNIT7
# Generate Prometheus Snapshot on Meas Node
### This notebook creates a Prometheus snapshot on the measurement node and helps you copy it to a backup destinaton

### Prerequisites
- Slice needs to be created: [Prepare a Slice](./knit7_prepare_a_slice_via_images.ipynb)
- Slice needs to be instrumentized: [Instrumentize a Slice](./knit7_instrumentize_a_slice.ipynb)
---

## Setup

#### Set Slice Name

In [ ]:
slice_name = "MyMonitoredSlice"

#### Import MF data transfer tools and create a new Prom export tools object with slice name

In [ ]:
from mflib.data_transfer import PrometheusExporter
prom_tools = PrometheusExporter(slice_name=slice_name)

#### Display the current Grafana tunnel

In [ ]:
# Grafana SSH Tunnel Command
# prom_tools.grafana_tunnel_local_port = 10010 # optionally change the port
print(prom_tools.grafana_tunnel)

print(f"Browse to https://localhost:{prom_tools.grafana_tunnel_local_port}/grafana/dashboards?query=%2A")

# The grafana_manager service was created by the prom_tools.instrumentize call. (prom_tools is an mflib object).
# Get access info for Grafana by using the mflib.info call to the grafana_manager.
# Create a dictionary to pass to the service.
data = {}
# Set the info you want to get.
data["get"] = ["admin_password"]
# Call info using service name and data dictionary.
info_results = prom_tools.info("grafana_manager", data)
print(info_results)

---
## Create a Prometheus Snapshot

#### Create a Prometheus snapshot

In [ ]:
# Get prometheus admin credentials so we can create snapshots
data={}
data["get"] = ["ht_user", "ht_password"]
prom_credentials = prom_tools.info("prometheus", data)

In [ ]:
snapshot_name = prom_tools.create_snapshot(prom_credentials["ht_user"],prom_credentials["ht_password"])
print(f"Snapshot name : {snapshot_name}")

#### Export snapshot to Measurement node as tar file

In [ ]:
prom_tools.export_snapshot_tar(snapshot_name)

#### Check that the snapshot file is in the directory on the Measurement Node

In [ ]:
prom_tools.view_snapshot_directory()

---
## Copy snapshot off Measurement node

### Option 1: Copy to a local device with SCP

##### Prerequisite
- For this step, you need your local ssh environment configured ([Configure Environment](./configure.ipynb))

##### Example SCP (Secure Copy) command to download snapshot file from VM

scp -F ~/.ssh/fabric/ssh_config -i ~/.ssh/fabric/slice_key ubuntu@\[2620:0:c80:1001:f816:3eff:fe81:80cc]:/home/mfuser/services/elk/files/snapshots/snapshot.tar .

##### Fill in the variables below then run to generate your custom SCP command

In [ ]:
# Path of directory to place your snapshot file
directory_path = "/your/path/here"

print(prom_tools.generate_scp_download_command(snapshot_name, directory_path))

##### More info
Fabric guide for transferring data directly from VM to local machine:
https://learn.fabric-testbed.net/knowledge-base/transferring-data-to-and-from-your-vms/

### Option 2: Copy to Jupyter Hub server

##### **CAUTION** The JH servers have very limited storage. If you are doing this method, make sure you do not download large or multiple snapshot files.
##### Fill in the variables below then run to download file to JH server

In [ ]:
# Snapshot will be in data_transfer_service/snapshots directory
prom_tools.meas_node.download_file(f"./snapshots/{snapshot_name}.tar", f"/home/mfuser/services/prometheus/files/snapshots/{snapshot_name}.tar")

---

# Export complete

For import instructions, refer back to [Start Here](./start_here.ipynb)